In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import shutil
from typing import Dict, Tuple

In [ ]:
df = pd.read_csv('loco_11_corr.tsv', sep='\t')
data_for_visualisation = pd.DataFrame({
        'loco_11.tu17r1': df['loco_11.tu17r1'],
        'loco_11.tu17l2': df['loco_11.tu17l2'],
        'loco_11.tu17r2': df['loco_11.tu17r2'],
        'loco_11.tu17l3': df['loco_11.tu17l3'],
        ''
        '': df['loco_11.tu17r3']
    })

In [4]:
def get_user_limits(columns: list) -> Dict[str, Tuple[float, float]]:
    """Интерактивный ввод границ для столбцов с валидацией"""
    limits = {}
    print("Введите границы (нижняя верхняя) или оставьте пустым для авторасчета\n")
    
    for col in columns:
        while True:
            user_input = input(f"{col}: ").replace(',', ' ').strip()
            if not user_input:  # Авторасчет при пустом вводе
                limits[col] = None
                break
            
            try:
                lower, upper = map(float, user_input.split())
                if lower >= upper:
                    print("Ошибка: нижняя граница должна быть меньше верхней!")
                    continue
                limits[col] = (lower, upper)
                break
            except:
                print("Некорректный ввод! Пример: 0 100 или 50.5, 200.75")
    
    return limits

class BoxplotVisualizer:
    """Генератор боксплотов и гистограмм с автоматической нормализацией"""
    
    def __init__(self, data: pd.DataFrame):
        self.data = data
        self.user_limits = get_user_limits(data.columns.tolist())
        self.output_folder = "visualizations"
        self._prepare_output_folder()

    def _prepare_output_folder(self):
        """Очистка и создание папки для результатов"""
        shutil.rmtree(self.output_folder, ignore_errors=True)
        os.makedirs(self.output_folder)

    def _calculate_statistics(self, series: pd.Series) -> dict:
        """Расчет статистик и границ по методу Тьюки"""
        if series.name not in self.data.columns:
            raise ValueError(f"Столбец {series.name} не найден в DataFrame")
        
        stats = {
            'Q1': np.percentile(series, 25),
            'Q3': np.percentile(series, 75),
            'median': series.median(),
            'lower': None,
            'upper': None,
            'outliers': 0
        }
        
        if col_limits := self.user_limits.get(series.name):
            stats['lower'], stats['upper'] = col_limits
        else:
            IQR = stats['Q3'] - stats['Q1']
            stats['lower'] = max(stats['Q1'] - 1.5*IQR, series.min())
            stats['upper'] = min(stats['Q3'] + 1.5*IQR, series.max())
        
        stats['outliers'] = ((series < stats['lower']) | (series > stats['upper'])).sum()
        return stats

    def _determine_scale(self, series: pd.Series, stats: dict) -> str:
        """Автоматический выбор шкалы для визуализации"""
        data_range = stats['upper'] - stats['lower']
        if (series.max() - stats['upper']) > 10*data_range or (stats['lower'] - series.min()) > 10*data_range:
            return 'log' if stats['lower'] > 0 else 'symlog'
        return 'linear'

    def _create_boxplot(self, series: pd.Series, stats: dict):
        """Построение нормализованного боксплота"""
        plt.figure(figsize=(12, 7))
        ax = plt.gca()
        
        # Настройка стиля элементов
        box_style = {
            'patch_artist': True,
            'boxprops': dict(facecolor='#8DA0CB', linewidth=2),
            'whiskerprops': dict(linewidth=1.5, color='#666666'),
            'medianprops': dict(color='#E7298A', linewidth=3),
            'flierprops': dict(
                marker='o', 
                markerfacecolor='#FC8D62', 
                markersize=5,
                alpha=0.5
            )
        }
        
        # Построение графика с автоматическим масштабированием
        scale_type = self._determine_scale(series, stats)
        ax.boxplot(series.dropna(), **box_style)
        ax.set_yscale(scale_type)
        
        # Добавление лимитов и аннотаций
        ax.axhline(stats['lower'], color='#66C2A5', linestyle='--', linewidth=2)
        ax.axhline(stats['upper'], color='#66C2A5', linestyle='--', linewidth=2)
        
        # Текст статистики с адаптивным позиционированием
        stats_text = (f"Медиана: {stats['median']:.2f}\nQ1: {stats['Q1']:.2f}\n"
                     f"Q3: {stats['Q3']:.2f}\nВыбросы: {stats['outliers']}")
        plt.text(1.08, 0.5, stats_text, transform=ax.transAxes,
                bbox=dict(facecolor='white', alpha=0.8), fontsize=12)
        
        plt.subplots_adjust(right=0.85)  # Место для аннотации
        ax.set_title(f"Box-plot ({scale_type} шкала): {series.name}", fontsize=14)
        plt.savefig(f"visualizations/boxplot_{series.name}.svg", bbox_inches='tight')
        plt.close()

    def _create_histogram(self, series: pd.Series, stats: dict):
        """Гистограмма с выделением выбросов"""
        plt.figure(figsize=(10, 7))
        
        # Разделение данных на основные и выбросы
        main_data = series[(series >= stats['lower']) & (series <= stats['upper'])]
        outliers = series[(series < stats['lower']) | (series > stats['upper'])]
        
        # Настройка бинов для лучшей визуализации
        bins = np.linspace(stats['lower'], stats['upper'], 50)
        
        plt.hist(main_data, bins=bins, color='skyblue', edgecolor='black', label='Основные данные')
        plt.hist(outliers, bins=50, color='red', alpha=0.5, label='Выбросы')
        
        plt.axvline(stats['lower'], color='green', linestyle='--', linewidth=2)
        plt.axvline(stats['upper'], color='green', linestyle='--', linewidth=2)
        plt.title(f"Histogram: {series.name}\nВыбросы: {stats['outliers']}", fontsize=14)
        plt.legend()
        plt.savefig(f"visualizations/hist_{series.name}.svg")
        plt.close()

    def generate_visualizations(self):
        """Основной цикл генерации графиков"""
        for col in self.data.columns:
            try:
                series = self.data[col].dropna()
                if series.empty:
                    print(f"Пропуск пустого столбца: {col}")
                    continue
                    
                stats = self._calculate_statistics(series)
                self._create_boxplot(series, stats)
                self._create_histogram(series, stats)
            except Exception as e:
                print(f"Ошибка при обработке столбца {col}: {str(e)}")
                continue

if __name__ == "__main__":
    # Инициализация данных из существующего DataFrame
    data = df.copy()
    
    # Запуск процесса визуализации
    visualizer = BoxplotVisualizer(data)
    visualizer.generate_visualizations()
    print("Все графики сохранены в папку 'visualizations'")

Введите границы (нижняя верхняя) или оставьте пустым для авторасчета

Некорректный ввод! Пример: 0 100 или 50.5, 200.75
Некорректный ввод! Пример: 0 100 или 50.5, 200.75
Ошибка при обработке столбца loco_11.dan_type: ufunc 'subtract' did not contain a loop with signature matching types (dtype('<U7'), dtype('<U7')) -> None
Ошибка при обработке столбца loco_11.zamer_type: ufunc 'subtract' did not contain a loop with signature matching types (dtype('<U13'), dtype('<U13')) -> None
Ошибка при обработке столбца loco_11.repair_date: ufunc 'subtract' did not contain a loop with signature matching types (dtype('<U19'), dtype('<U19')) -> None


KeyboardInterrupt: 

In [ ]:
class ScatterPlotGenerator:
    def __init__(self, dataframe):
        self.df = dataframe
        self.output_dir = "scatter_plots"
        self._create_output_dir()
        
    def _create_output_dir(self):
        """Создает директорию для сохранения графиков, если ее нет"""
        os.makedirs(self.output_dir, exist_ok=True)

    def _validate_columns(self, col1, col2):
        """Проверяет существование столбцов в DataFrame"""
        return all(col in self.df.columns for col in [col1, col2])

    def _get_numeric_limits(self, prompt, default_min, default_max):
        """Запрашивает и валидирует границы для оси"""
        while True:
            user_input = input(prompt)
            if not user_input:
                return default_min, default_max
            
            try:
                parts = list(map(float, user_input.split()))
                if len(parts) != 2 or parts[0] >= parts[1]:
                    raise ValueError
                return parts[0], parts[1]
            except:
                print("Ошибка формата! Используйте: мин макс (например: 50 200)")

    def _detect_outliers(self, df, col_x, col_y):
        """Определяет выбросы с использованием IQR"""
        Q1_x = df[col_x].quantile(0.25)
        Q3_x = df[col_x].quantile(0.75)
        IQR_x = Q3_x - Q1_x
        lower_x = Q1_x - 1.5 * IQR_x
        upper_x = Q3_x + 1.5 * IQR_x

        Q1_y = df[col_y].quantile(0.25)
        Q3_y = df[col_y].quantile(0.75)
        IQR_y = Q3_y - Q1_y
        lower_y = Q1_y - 1.5 * IQR_y
        upper_y = Q3_y + 1.5 * IQR_y

        outliers_mask = (
            (df[col_x] < lower_x) | (df[col_x] > upper_x) |
            (df[col_y] < lower_y) | (df[col_y] > upper_y)
        )
        return outliers_mask

    def generate_plot(self):
        """Основной метод генерации графика"""
        col_x = input("Введите название столбца для оси X: ").strip()
        col_y = input("Введите название столбца для оси Y: ").strip()

        if not self._validate_columns(col_x, col_y):
            print("Ошибка: один или оба указанных столбца не существуют в DataFrame")
            return

        clean_df = self.df[[col_x, col_y]].dropna()
        if clean_df.empty:
            print("Нет данных для построения графика после удаления NaN-значений")
            return

        x_data_initial = clean_df[col_x]
        y_data_initial = clean_df[col_y]

        x_min, x_max = self._get_numeric_limits(
            f"Границы для {col_x} [мин макс] (Enter для авто): ",
            x_data_initial.min(),
            x_data_initial.max()
        )

        y_min, y_max = self._get_numeric_limits(
            f"Границы для {col_y} [мин макс] (Enter для авто): ",
            y_data_initial.min(),
            y_data_initial.max()
        )

        # Определение выбросов
        outliers_mask = self._detect_outliers(clean_df, col_x, col_y)
        df_outliers = clean_df[outliers_mask]
        df_non_outliers = clean_df[~outliers_mask]

        original_total = len(clean_df)
        target_total = max(1, original_total // 20)  # Целевое количество точек

        available_for_non_outliers = max(0, target_total - len(df_outliers))

        # Формируем прореженный датасет
        if available_for_non_outliers <= 0:
            clean_df_subsampled = df_outliers.sample(n=min(target_total, len(df_outliers)), random_state=42)
        elif len(df_non_outliers) == 0:
            clean_df_subsampled = df_outliers
        else:
            if available_for_non_outliers >= len(df_non_outliers):
                clean_df_subsampled = pd.concat([df_non_outliers, df_outliers])
            else:
                sampled_non_outliers = df_non_outliers.sample(n=available_for_non_outliers, random_state=42)
                clean_df_subsampled = pd.concat([sampled_non_outliers, df_outliers])

        # Обрезаем до целевого количества
        clean_df_subsampled = clean_df_subsampled.head(target_total)

        if clean_df_subsampled.empty:
            print("Нет данных для построения графика после субсэмплинга")
            return

        x_data = clean_df_subsampled[col_x]
        y_data = clean_df_subsampled[col_y]

        # Создание графика
        plt.figure(figsize=(12, 8))
        plt.scatter(
            x_data, 
            y_data,
            alpha=0.6,
            edgecolor='w',
            linewidth=0.5,
            s=10  # Уменьшаем размер точек для лучшей визуализации
        )

        plt.xlim(x_min, x_max)
        plt.ylim(y_min, y_max)
        
        plt.axvline(x_min, color='r', linestyle='--', alpha=0.7)
        plt.axvline(x_max, color='r', linestyle='--', alpha=0.7)
        plt.axhline(y_min, color='r', linestyle='--', alpha=0.7)
        plt.axhline(y_max, color='r', linestyle='--', alpha=0.7)

        plt.title(f"Scatter Plot: {col_x} vs {col_y}\nLimits: X({x_min}-{x_max}), Y({y_min}-{y_max})")
        plt.xlabel(col_x)
        plt.ylabel(col_y)
        plt.grid(alpha=0.2)

        filename = f"{col_x}_vs_{col_y}.svg"
        plt.savefig(
            os.path.join(self.output_dir, filename), 
            format='svg', 
            bbox_inches='tight'
        )
        print(f"График сохранен как: {os.path.join(self.output_dir, filename)}")
        plt.close()

# Пример использования
if __name__ == "__main__":
    # Пример DataFrame
    df = df
    
    plotter = ScatterPlotGenerator(df)
    plotter.generate_plot()